In [9]:
# !pip install auto-sklearn==0.14.8 --user
# !pip install lz4==4.0.0

In [1]:
from pprint import pprint
import time

import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import numpy as np
import subprocess

In [2]:
autosklearn_metrics = {'roc_auc' : autosklearn.metrics.roc_auc}

autosklearn_metrics['roc_auc']


roc_auc

In [3]:
import importlib

metric = 'log_loss'

sk_metric = 'autosklearn.metrics.{}'.format(metric)

sk_metric - importlib.import_module(sk_metric)

print(type(sk_metric))
type(autosklearn.metrics.log_loss)

ModuleNotFoundError: No module named 'autosklearn.metrics.log_loss'

In [3]:
# import lz4

# lz4.__version__

In [4]:
#load data from Domino project directory
#load and clean the data----------------------

#load data from Domino project directory
hd_data = pd.read_csv("./data/raw/heart.csv")

X = hd_data.drop('target', axis=1)
y = hd_data.target 

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

hd_data.info()
hd_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
%%time

scorer = autosklearn.metrics.log_loss

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    ensemble_nbest = 5,
    resampling_strategy= 'cv',
    resampling_strategy_arguments = {'folds': 5},
    seed = 42,
    metric=scorer,
    scoring_functions = [autosklearn.metrics.log_loss,
                         autosklearn.metrics.roc_auc,
                         autosklearn.metrics.f1],
    tmp_folder='/tmp/autosklearn_classification_example_tmp',
    n_jobs=-1, dask_client=client

    
)

automl.fit(X_train, y_train, dataset_name='heat_disease')


[ERROR] [2022-09-26 16:42:25,676:Client-AutoML(42):heat_disease] [Errno 2] No such file or directory: '/tmp/autosklearn_classification_example_tmp/.auto-sklearn/datamanager.pkl'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/opt/conda/lib/python3.8/site-packages/autosklearn/smbo.py", line 552, in run_smbo
    smac.optimize()
  File "/opt/conda/lib/python3.8/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/opt/conda/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 316, in run
    self._incorporate_run_results(run_info, result, time_left)
  File "/opt/conda/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 551, in _incorporate_run_results
    response = callback(
  File "/opt/conda/lib/python3.8/site-packages/autosklearn/ensemble_building/manager.py", line 170, in __call__
    self.build_ensemble(

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/autosklearn_classification_example_tmp/.auto-sklearn/datamanager.pkl'

In [8]:
print(sklearn.__version__)

0.24.2


In [ ]:
model_type = [clf['classifier:__choice__'] for clf in automl.cv_results_['params']]

automl_results = pd.DataFrame.from_dict(automl.cv_results_, orient='columns')

automl_results = automl_results[['mean_test_score', 'metric_log_loss',
                                 'metric_roc_auc', 'metric_f1',
                                 'mean_fit_time', 'params', 'rank_test_scores']]

automl_results['model_type'] = model_type
                                
automl_results = automl_results.sort_values(by='rank_test_scores', ascending=True)

automl_results.head(20)


In [4]:
from platform import python_version 
print(python_version())

3.8.12


In [5]:
from dask.distributed import Client
import os

service_host = os.environ["DASK_SCHEDULER_SERVICE_HOST"]
service_port = os.environ["DASK_SCHEDULER_SERVICE_PORT"]
client = Client(address=f"{service_host}:{service_port}")

# you should now be connected to the cluster
# Dashboard link from the client object is clickable but will not route in Domino
# Use the embedded Dask Web UI tab instead

# You can also expand the "Scheduler Info" in the client object to see some cluster information
client

<Client: 'tcp://10.0.111.15:8786' processes=1 threads=1, memory=4.00 GiB>

In [8]:
%%time

scorer = autosklearn.metrics.log_loss

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=3072,
    # ensemble_nbest = 5,
    resampling_strategy= 'cv',
    resampling_strategy_arguments = {'folds': 5},
    seed = 42,
    metric=scorer,
    scoring_functions = [autosklearn.metrics.log_loss,
                         autosklearn.metrics.roc_auc,
                         autosklearn.metrics.f1],
    tmp_folder='/tmp/autosklearn_classification_example_tmp',
    
    #turn on parallelization
    n_jobs=-1,
    dask_client=client,
)


automl.fit(X_train, y_train, dataset_name='heat_disease')

[ERROR] [2022-09-26 16:42:49,584:Client-AutoML(42):heat_disease] [Errno 2] No such file or directory: '/tmp/autosklearn_classification_example_tmp/.auto-sklearn/datamanager.pkl'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/opt/conda/lib/python3.8/site-packages/autosklearn/smbo.py", line 552, in run_smbo
    smac.optimize()
  File "/opt/conda/lib/python3.8/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/opt/conda/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 316, in run
    self._incorporate_run_results(run_info, result, time_left)
  File "/opt/conda/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 551, in _incorporate_run_results
    response = callback(
  File "/opt/conda/lib/python3.8/site-packages/autosklearn/ensemble_building/manager.py", line 170, in __call__
    self.build_ensemble(

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/autosklearn_classification_example_tmp/.auto-sklearn/datamanager.pkl'

In [7]:
%%time

#build the auto-sklearn routine
automl_hd_ohe_p = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    disable_evaluator_output=False,
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67},
    
    #turn on parallelization
    n_jobs=4,
    seed=5,
)

#call it
automl_hd_ohe_p.fit(X_train, y_train, dataset_name='heart_disease')


[WARNING] [2022-08-31 22:25:30,806:Client-AutoML(5):heart_disease] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
[ERROR] [2022-08-31 22:26:25,027:asyncio.events] 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/distributed/client.py", line 1246, in _reconnect
    await self._ensure_connected(timeout=timeout)
  File "/opt/conda/lib/python3.8/site-packages/distributed/client.py", line 1276, in _ensure_connected
    comm = await connect(
  File "/opt/conda/lib/python3.8/site-packages/distributed/comm/core.py", line 315, in connect
    await asyncio.sleep(backoff)
  File "/opt/conda/lib/python3.8/asyncio/tasks.py", line 659, in sleep
    return await future
asyncio.exceptions.CancelledError
[ERROR] [2022-08-31 22:26:25,029:asyncio.events] 
Traceback (most recent call last):
  File "/op

AutoSklearnClassifier(n_jobs=4, per_run_time_limit=30,
                      resampling_strategy_arguments={'train_size': 0.67},
                      seed=5, time_left_for_this_task=60)

In [ ]:
model_type = [clf['classifier:__choice__'] for clf in automl.cv_results_['params']]

automl_results = pd.DataFrame.from_dict(automl.cv_results_, orient='columns')

automl_results = automl_results[['mean_test_score', 'metric_log_loss',
                                 'metric_roc_auc', 'metric_f1',
                                 'mean_fit_time', 'params', 'rank_test_scores']]

automl_results['model_type'] = model_type
                                
automl_results = automl_results.sort_values(by='rank_test_scores', ascending=True)

automl_results.head(20)

In [ ]:
autosklearn.__version__